In [1]:
!pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import v2
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [3]:
use_s3 = 0

EPOCH = 5
BATCH_SIZE = 32

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5

In [4]:
import wandb
# This is secret and shouldn't be checked into version control
WANDB_API_KEY="349b387de860518b8d586052fab2caa41953cb97"
wandb.login(key=WANDB_API_KEY)
# Name and notes optional
WANDB_NAME="my wandb name"
WANDB_NOTES="my wandb notes"
# wandb.init(mode="disabled")
run = wandb.init(
    # Set the project where this run will be logged
    project="cats-vs-dogs",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCH,
        "batch_size": BATCH_SIZE,
        "weight_decay": WEIGHT_DECAY
    },
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tzu-chun-huang-tw (team-tc-huang). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231224_134448-eob781ul
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fresh-lion-11
wandb: ⭐️ View project at https://wandb.ai/team-tc-huang/cats-vs-dogs
wandb: 🚀 View run at https://wandb.ai/team-tc-huang/cats-vs-dogs/runs/eob781ul


In [5]:
# %%writefile ./scripts/train.py
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()

    # Get the name of the GPU(s)
    gpu_names = [torch.cuda.get_device_name(i) for i in range(num_gpus)]

    print(f"GPU(s) available: {num_gpus}")
    for i, gpu_name in enumerate(gpu_names):
        print(f"GPU {i}: {gpu_name}")
else:
    print("No GPU available. PyTorch will run on CPU.")

GPU(s) available: 1
GPU 0: Tesla P100-PCIE-16GB


In [6]:
from transformers import ConvNextImageProcessor, ResNetForImageClassification

image_processor = ConvNextImageProcessor.from_pretrained("microsoft/resnet-50")
# label2id = {"Cat": 0, "Dog": 1}
# id2label = {0: "Cat", 1: "Dog"}

model = ResNetForImageClassification.from_pretrained(
    "microsoft/resnet-50",
    ignore_mismatched_sizes=True,
    num_labels=1
     # num_labels=len(label2id),
     # id2label=id2label,
     # label2id=label2id
)
model.classifier[1] = nn.Linear(in_features=2048, out_features=1, bias=True)
model.classifier.add_module("2", nn.Sigmoid())

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# print(f"device: {device}")
# model = model.to(device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([1, 2048]) in the model instantiated
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from PIL import Image

training_dataset_dir = "/kaggle/input/cats-vs-dogs/dataset/training_dataset"
validation_dataset_dir = "/kaggle/input/cats-vs-dogs/dataset/validation_dataset"

class CatDogDataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        
        cat_image_dir = os.path.join(dataset_dir, 'Cat', '*.jpg')
        cat_image_paths = glob.glob(cat_image_dir)
        print(f"cat_image_paths: {cat_image_dir}, len: {len(cat_image_paths)}")
        cat_image_labels = torch.zeros(size=(len(cat_image_paths),))
        
        dog_image_dir = os.path.join(dataset_dir, 'Dog', '*.jpg')
        dog_image_paths = glob.glob(dog_image_dir)
        print(f"dog_image_paths: {dog_image_dir}, len: {len(dog_image_paths)}")
        dog_image_labels = torch.ones(size=(len(dog_image_paths),))
        
        self.image_paths = np.concatenate((cat_image_paths, dog_image_paths))
        self.image_labels = np.concatenate(( cat_image_labels, dog_image_labels))

        self.transform = transform
        
         
    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = np.array(image)
            iamge = self.transform(image)
            image = Image.fromarray(image)
            
        label = self.image_labels[idx]
        label = torch.from_numpy(np.array([label]))
#         label = torch.from_numpy(np.array([[label]))
        return {'image': image,
                 'label': label}
    
    def label_dict(self):
        return {0:'Cat', 1:'Dog'}
    
    def get_cat_dog_image(self):
        cat_image = read_image(self.image_paths[0])
        dog_image = read_image(self.image_paths[-1])
        return cat_image, dog_image

train_transforms = v2.Compose(
    [
        v2.RandomErasing()
    ]
)


train_dataset = CatDogDataset(training_dataset_dir, transform=None)
validation_dataset = CatDogDataset(validation_dataset_dir, transform=None)

cat_image_paths: /kaggle/input/cats-vs-dogs/dataset/training_dataset/Cat/*.jpg, len: 5412
dog_image_paths: /kaggle/input/cats-vs-dogs/dataset/training_dataset/Dog/*.jpg, len: 10788
cat_image_paths: /kaggle/input/cats-vs-dogs/dataset/validation_dataset/Cat/*.jpg, len: 588
dog_image_paths: /kaggle/input/cats-vs-dogs/dataset/validation_dataset/Dog/*.jpg, len: 1212


In [8]:
# from torch.utils.data import Subset

# # Assuming train_dataset is a PyTorch Dataset
# indices = range(0, 64)  # Define the range of indices you want
# train_dataset = Subset(train_dataset, indices)
# validation_dataset = Subset(validation_dataset, indices)

In [9]:
from datetime import datetime
from transformers import TrainingArguments, Trainer
import evaluate

print(f"epoch: {EPOCH}")
print(f"batch_size: {BATCH_SIZE}")

TRAIN_DATASET = train_dataset
VALIDATION_DATASET = validation_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")
time_now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

my_batch = None

training_args = TrainingArguments(
    output_dir=f"detr-resnet-50_{time_now}",
    per_device_train_batch_size=int(BATCH_SIZE),
    per_device_eval_batch_size=int(BATCH_SIZE),
    num_train_epochs=int(EPOCH),
    save_strategy = "epoch",
    evaluation_strategy="epoch",
    fp16=False,
    # save_steps=200,
    logging_steps=8,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=2,
    seed=42,
    load_best_model_at_end = True,
    # metric_for_best_model="loss",
    remove_unused_columns=False,
    torch_compile=False,
#     dataloader_pin_memory=True,
#     dataloader_num_workers=0,
    push_to_hub=False,
)


def collate_fn(batch):
    image_list = [item['image'] for item in batch]
    label_list = [item['label'] for item in batch]
    pixel_values = image_processor(image_list, return_tensors="pt").pixel_values
    batch_ = {}
    batch_["pixel_values"] = pixel_values#.to(device)
    batch_["labels"] = torch.stack(label_list)#.to(device)
#     print(f"labels in batch: {batch_['labels'].shape}")
    return batch_


accuracy = evaluate.load("accuracy")
# correct = 0
# count = 0

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
#     print(f"predictions: {predictions.shape}")
#     print(f"labels: {labels.shape}")
#     predictions = np.argmax(predictions, axis=1)
#     threshold = 0.5
    predictions = np.round(predictions)
#     print(f"predictions processed: {predictions.shape, predictions}")
#     print(f"labels processed: {labels.shape, labels}")
          
    result = accuracy.compute(predictions=predictions, references=labels)
#     print(f"result: {result}")
#     global count, correct
#     matching_count = np.sum(predictions == labels).item()
    
#     correct += matching_count
#     count += len(labels)
    
    return result

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=TRAIN_DATASET,
    eval_dataset=VALIDATION_DATASET,
    compute_metrics = compute_metrics
    # tokenizer=image_processor,
)

trainer.train()
trainer.save_model(f"detr-resnet-50_{time_now}_best1")

epoch: 5
batch_size: 32
device: cuda


Epoch,Training Loss,Validation Loss,Accuracy
1,0.010100,0.020915,0.972222
2,0.014900,0.014691,0.981111
3,0.003900,0.009973,0.984444
4,0.002000,0.010335,0.986111
5,0.000100,0.007867,0.988889


/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [10]:
# a = np.array([[0.],
#        [1.],
#        [0.],
#        [0.]])
# b = np.array([[1.],
#        [0.],
#        [0.],
#        [0.]])
# accuracy.compute(predictions=b, references=a)

In [11]:
# c = np.array([[0.6],
#        [0.3],
#        [0.],
#        [0.]])
# np.round(c)

In [12]:
# print(f"Accuracy: {correct / count}")

In [13]:
# global count, correct

In [14]:
# from IPython.display import display
# from tqdm import trange

# correct = 0
# model.to(device)
# for i in trange(len(validation_dataset)):
#     image = validation_dataset[i]['image']
#     label = validation_dataset[i]['label'].to(device)
#     # display(image)
#     # print(f"label: {inference_dataset.label_dict()[int(label)]}")
#     inputs = image_processor(image, return_tensors="pt")
#     inputs['pixel_values'] = inputs['pixel_values'].half().to(device)
#     with torch.no_grad():
#         logits = model(**inputs).logits
#     prediction = torch.round(logits)
#     # print(f"prediction: {inference_dataset.label_dict()[int(prediction)]}")
#     if label == prediction:
#         correct += 1

# print(f"correct: {correct}, total: {len(validation_dataset)}, accuracy: {correct/len(validation_dataset)}")